In [2]:
import os
import sys
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [3]:
import figure_setup as fs

# Get ZTF fit

In [4]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [5]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

fink_colors = ["#15284F", "#F5622E"]


In [6]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [7]:
data.columns

Index(['ssnamenr', 'HG_chi2red', 'HG_status', 'HG_fit', 'HG_rms', 'HG_rms_g',
       'HG_rms_r', 'HG_median_error_phot', 'HG_median_error_phot_1',
       'HG_median_error_phot_2', 'HG_H_g', 'HG_dH_g', 'HG_G_g', 'HG_dG_g',
       'HG_H_r', 'HG_dH_r', 'HG_G_r', 'HG_dG_r', 'HG_flag', 'HG1G2_chi2red',
       'HG1G2_status', 'HG1G2_fit', 'HG1G2_rms', 'HG1G2_rms_g', 'HG1G2_rms_r',
       'HG1G2_median_error_phot', 'HG1G2_median_error_phot_1',
       'HG1G2_median_error_phot_2', 'HG1G2_H_g', 'HG1G2_dH_g', 'HG1G2_G1_g',
       'HG1G2_dG1_g', 'HG1G2_G2_g', 'HG1G2_dG2_g', 'HG1G2_H_r', 'HG1G2_dH_r',
       'HG1G2_G1_r', 'HG1G2_dG1_r', 'HG1G2_G2_r', 'HG1G2_dG2_r', 'HG1G2_flag',
       'SHG1G2_chi2red', 'SHG1G2_min_cos_lambda', 'SHG1G2_mean_cos_lambda',
       'SHG1G2_max_cos_lambda', 'SHG1G2_status', 'SHG1G2_fit', 'SHG1G2_rms',
       'SHG1G2_rms_g', 'SHG1G2_rms_r', 'SHG1G2_median_error_phot',
       'SHG1G2_median_error_phot_1', 'SHG1G2_median_error_phot_2', 'n_obs',
       'n_obs_g', 'n_obs_r', 

In [8]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [9]:
# Not useful anymore: to find the best columns
# bft.columns[ bft.columns.str.contains(r'^(?=.*color)(?=.*value)') ]

In [10]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.c-o.color.value",
    "colors.c-o.color.error.min",
    "colors.c-o.color.error.max",
    "colors.c-o.facility",
    "colors.c-o.observer",
    "colors.c-o.epoch",
    "colors.c-o.delta_time",
    "colors.c-o.id_filter_1",
    "colors.c-o.id_filter_2",
    "colors.c-o.phot_sys",
    "colors.c-o.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "colors.g-i.color.value",
    "colors.g-i.color.error.min",
    "colors.g-i.color.error.max",
    "colors.g-i.facility",
    "colors.g-i.observer",
    "colors.g-i.epoch",
    "colors.g-i.delta_time",
    "colors.g-i.id_filter_1",
    "colors.g-i.id_filter_2",
    "colors.g-i.phot_sys",
    "colors.g-i.technique",
    "colors.i-z.color.value",
    "colors.i-z.color.error.min",
    "colors.i-z.color.error.max",
    "colors.i-z.facility",
    "colors.i-z.observer",
    "colors.i-z.epoch",
    "colors.i-z.delta_time",
    "colors.i-z.id_filter_1",
    "colors.i-z.id_filter_2",
    "colors.i-z.phot_sys",
    "colors.i-z.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]

bft = pd.read_parquet(bft_file, columns=cols)

In [11]:
data = data.merge(bft, left_on="name", right_on="sso_name", how="left")

In [12]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



In [13]:
maskFINK = maskFINK_union

# Color

In [14]:
def color_conv_ztf_panstarrs(filter, g_minus_r):
    """
    Return mag_ztf - mag_PS in g or r
    Medford 2020
    DOI 10.3847/2515-5172/ab7f3c
    """
    if filter == "g":
        return 0.055 * (g_minus_r) - 0.012
    else:
        return -0.087 * (g_minus_r) - 0.0035

In [15]:
def color_conv_panstarrs_sdss(filter, g_minus_i):
    """
    Return mag_PS - mag_SDSS in g or r
    Medford 2020
    DOI 10.3847/2515-5172/ab7f3c
    """
    if filter == "g":
        a0, a1, a2, a3 = -0.01808, -0.13595, 0.01941, -0.00183
    elif filter == "r":
        a0, a1, a2, a3 = -0.01836, -0.03577, 0.02612, -0.00558
    else:
        print("aie")

    return a0 + a1 * g_minus_i + a2 * g_minus_i**2 + a3 * g_minus_i**3

In [16]:
def color_conv_skymapper_sdss(color, value):
    """
    Return mag_skymapper - mag_sdss in g or r
    Medford 2020
    DOI 10.3847/2515-5172/ab7f3c
    """
    if color == "g-r":
        return 1.46534 * value + 0.027855
        
    if color == "g-i":
        return 1.171337 * value + 0.06685

    else:
        print("sm sdss aie")

# gr vs SDSS and SkyMapper - Hsplit

In [48]:
# ----------------------------------------------------------------------
fig, ax = plt.subplots(
    1,
    2,
    figsize=fs.figsize(0.5, aspect=1.8),
    # sharex=True,
    sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "top": 0.98,
        "bottom": 0.13,
        "left": 0.1,
        "right": 0.97,
    },
)


# ----------------------------------------------------------------------
# Parameters for histograms
r = [[0.05, 0.95], [0.05, 0.95]]
b = 30
xTxt, yTxt = 0.05, 0.875
lim_dH = 1e6 # no cut
vmin, vmax = 0, 150

r = [[0.2, 0.8], [0.2, 0.8]]
vmin, vmax = 0, 100
cmap = 'viridis'

# ----------------------------------------------------------------------
# SDSS

# Select sample
cond = (
    maskFINK
    & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
    & (data["colors.g-r.facility"] == "SDSS")
    & (data["colors.g-r.color.value"].notna())
    & (data["colors.g-i.facility"] == "SDSS")
    & (data["colors.g-i.color.value"].notna())
)

# Define x/y
x = data.loc[cond, "colors.g-r.color.value"]
y = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
x_gi = data.loc[cond, "colors.g-i.color.value"]
ax[0].text(xTxt, yTxt, f"{len(data[cond]):,d} SSOs", transform=ax[0].transAxes, color='white')

# Convert to PanSTARRS
dmag_g = color_conv_ztf_panstarrs("g", y)
dmag_r = color_conv_ztf_panstarrs("r", y)
y2 = y - (dmag_g - dmag_r)

dmag_g = color_conv_panstarrs_sdss("g", x_gi)
dmag_r = color_conv_panstarrs_sdss("r", x_gi)
x2 = x + (dmag_g - dmag_r)

# Plot 2d Histo
_, _, _, im_sdss = ax[0].hist2d(
    x2, y2, range=r, bins=b, cmap=cmap, vmin=vmin, vmax=vmax, rasterized=True
)

# Colorbar
axins = inset_axes(
    ax[0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="lower center",
)
cbar_sdss = fig.colorbar(im_sdss, cax=axins, orientation="horizontal")
cbar_sdss.ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False, labelcolor='white')

# ----------------------------------------------------------------------
# SkyMapper
# Select sample
cond = (
    maskFINK
    & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
    & (data["colors.g-r.facility"] == "SMSS")
    & (data["colors.g-r.color.value"].notna())
)

# Define x/y
x = data.loc[cond, "colors.g-r.color.value"]
y = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
ax[1].text(xTxt, yTxt, f"{len(data[cond]):,d} SSOs", transform=ax[1].transAxes, color='white')

# Convert to PanSTARRS
dmag_g = color_conv_ztf_panstarrs("g", y)
dmag_r = color_conv_ztf_panstarrs("r", y)
y2 = y - (dmag_g - dmag_r)

x_sdss = color_conv_skymapper_sdss("g-r", x)
dmag_g = color_conv_panstarrs_sdss("g", data.loc[cond, "colors.g-i.color.value"])
dmag_r = color_conv_panstarrs_sdss("r", data.loc[cond, "colors.g-i.color.value"])
x2 = x_sdss + (dmag_g - dmag_r)

# Plot histogram
ax[1].hist2d(
    x2, y2, range=r, bins=b, cmap=cmap, vmin=vmin, vmax=vmax, rasterized=True
)


# ----------------------------------------------------------------------
# Axes
ax[0].set_xlabel("SDSS $g$-$r$")
ax[1].set_xlabel("SkyMapper $g$-$r$")
ax[0].set_ylabel("ZTF H$_g$-H$_r$")
                 
xx = np.linspace(0, 1, num=10)
for a in ax[:].ravel():
    a.set_xlim(r[0][0], r[0][1])
    a.set_ylim(r[1][0], r[1][1])
    a.plot(xx, xx, linestyle="dashed", color="grey")
    a.grid()
    a.set_aspect("equal")

ax[0].set_xticks([0.2, 0.4, 0.6, 0.8])
ax[0].set_xticklabels([0.2, 0.4, 0.6, ''])


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_comparison.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_comparison.pgf"),
)
plt.close()

# gr vs iz

In [28]:
# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5, aspect=1.1),
    gridspec_kw={
        "top": 0.98,
        "bottom": 0.1,
        "left": 0.13,
        "right": 0.98,
    },
)

lim_dH = 0.25

# --------------------------------------------------------------------------------
# Plot objects with taxonomy
complexes = ["E", "V", "Q", "A", "S", "L", "K", "D", "M", "Z", "C", "B", "Ch", "P"]
complexes = ["V", "Q", "A", "S", "D", "Z", "C", "B"]
markers = ["v", "o", "^", ".", "d", "P", "s", "<"]

complexes = ["S", "C", "V", "D", "A", "B"]
markers = ['.', "s", "v", 'd', "^", "<"]
taxocolors = [u'#1f77b4','grey','red','brown','orange','cornflowerblue']

number_of_ssos = 0
for i, c in enumerate(complexes):
    # Select sample
    cond = (
        maskFINK
        & (data["taxonomy.complex"] == c)
        & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
        & (data["colors.i-z.facility"] == "SDSS")
        & (data["colors.i-z.color.value"].notna())
    )

    # Plot gr vs iz
    x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
    y = data.loc[cond, "colors.i-z.color.value"]
    ax.scatter(x, y, marker=markers[i],
               s=10, 
               color=taxocolors[i],
               label=f"{c} ({len(data[cond])})")
    

    # Place average letter
    ax.text(x.mean(), y.mean(), c, ha="center", va="center", color="black")
    
    number_of_ssos += len(x)

# --------------------------------------------------------------------------------
# Axes
ax.legend()
ax.set_xlabel("ZTF H$_g$-H$_r$")
ax.set_ylabel("SDSS $i$-$z$")
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.7, 0.3)
ax.grid()


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_griz.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_griz.pgf"),
)
plt.close()

In [29]:
number_of_ssos

3569

In [20]:
# import scipy.stats as st

# def contour_cloud(x, y, color, ax):
#     xmin, xmax = 0, 1 #np.min(x), np.max(x)
#     ymin, ymax = -0.8, 0.3 #np.min(y), np.max(y)
    
#     xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
#     positions = np.vstack([xx.ravel(), yy.ravel()])
#     values = np.vstack([x, y])
#     kernel = st.gaussian_kde(values)
#     f = np.reshape(kernel(positions).T, xx.shape)

#     m = np.amax(f)
#     cs = ax.contourf(xx, yy, f, levels=[m*0.35, m*0.5, m], colors=color, alpha=0.2)
#     ax.contour(cs, linestyles='--', linewidths=0.5, colors=color)

In [21]:
# # --------------------------------------------------------------------------------
# fig, ax = plt.subplots(
#     figsize=fs.figsize(0.5, aspect=1.1),
#     gridspec_kw={
#         "top": 0.98,
#         "bottom": 0.1,
#         "left": 0.13,
#         "right": 0.98,
#     },
# )

# lim_dH = 0.25

# # --------------------------------------------------------------------------------
# # Plot objects with taxonomy
# complexes = ["E", "V", "Q", "A", "S", "L", "K", "D", "M", "Z", "C", "B", "Ch", "P"]
# complexes = ["V", "Q", "A", "S", "D", "Z", "C", "B"]
# markers = ["v", "o", "^", ".", "d", "P", "s", "<"]

# complexes = ["S", "C", "V", "D", "A", "B"]
# markers = ['.', "s", "v", 'd', "^", "<"]
# taxocolors = [u'#1f77b4','grey','red','brown','orange','cornflowerblue']

# number_of_ssos = 0
# for i, c in enumerate(complexes):
#     # Select sample
#     cond = (
#         maskFINK
#         & (data["taxonomy.complex"] == c)
#         & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
#         & (data["colors.i-z.facility"] == "SDSS")
#         & (data["colors.i-z.color.value"].notna())
#     )

#     # Plot gr vs iz
#     x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
#     y = data.loc[cond, "colors.i-z.color.value"]
#     ax.scatter(x, y, marker=markers[i],
#                s=2, 
#                alpha=0.2,
#                color=taxocolors[i],
#                label=f"{c} ({len(data[cond])})")
    
# #     if (len(x) < 800) and (len(x) > 200):
#     contour_cloud(x, y, color=taxocolors[i], ax=ax)
    

#     # Place average letter
#     ax.text(x.mean(), y.mean(), c, ha="center", va="center", color="black")
    
#     number_of_ssos += len(x)

# # --------------------------------------------------------------------------------
# # Axes
# ax.legend()
# ax.set_xlabel("ZTF H$_g$-H$_r$")
# ax.set_ylabel("SDSS $i$-$z$")
# ax.set_xlim(-0.1, 1.1)
# ax.set_ylim(-0.7, 0.3)
# ax.grid()


# # --------------------------------------------------------------------------------
# fig.savefig(
#     os.path.join(data_fink, "gfx", "article", "color_griz2.png"),
#     facecolor="white",
# )

# fig.savefig(
#     os.path.join(data_fink, "gfx", "article", "color_griz2.pgf"),
# )
# plt.close()

# gr vs SDSS and SkyMapper - Vsplit

In [22]:
# ----------------------------------------------------------------------
fig, ax = plt.subplots(
    2,
    1,
    figsize=fs.figsize(0.5, aspect=1.2),
    sharex=True,
    sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "top": 0.98,
        "bottom": 0.1,
        "left": 0.1,
        "right": 0.98,
    },
)


# ----------------------------------------------------------------------
# Parameters for histograms
r = [[0.05, 0.95], [0.05, 0.95]]
b = 30
xTxt, yTxt = 0.05, 0.85
lim_dH = 7.7
vmin, vmax = 0, 150

# ----------------------------------------------------------------------
# SDSS

# Select sample
cond = (
    maskFINK
    & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
    & (data["colors.g-r.facility"] == "SDSS")
    & (data["colors.g-r.color.value"].notna())
    & (data["colors.g-i.facility"] == "SDSS")
    & (data["colors.g-i.color.value"].notna())
)

# Define x/y
x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
y = data.loc[cond, "colors.g-r.color.value"]
y_gi = data.loc[cond, "colors.g-i.color.value"]
ax[0].text(xTxt, yTxt, f"{len(data[cond]):,d} SSOs", transform=ax[0].transAxes)

# Convert to PanSTARRS
dmag_g = color_conv_ztf_panstarrs("g", x)
dmag_r = color_conv_ztf_panstarrs("r", x)
x2 = x - (dmag_g - dmag_r)

dmag_g = color_conv_panstarrs_sdss("g", y_gi)
dmag_r = color_conv_panstarrs_sdss("r", y_gi)
y2 = y + (dmag_g - dmag_r)

# Plot 2d Histo
_, _, _, im_sdss = ax[0].hist2d(
    x2, y2, range=r, bins=b, cmap="hot_r", vmin=vmin, vmax=vmax, rasterized=True
)

# Colorbar
axins = inset_axes(
    ax[0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="lower right",
)
cbar_sdss = fig.colorbar(im_sdss, cax=axins, orientation="horizontal")
cbar_sdss.ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)


# ----------------------------------------------------------------------
# SkyMapper
# Select sample
cond = (
    maskFINK
    & (np.sqrt(data["SHG1G2_dH_g"] ** 2 + data["SHG1G2_dH_r"] ** 2) < lim_dH)
    & (data["colors.g-r.facility"] == "SMSS")
    & (data["colors.g-r.color.value"].notna())
)

# Define x/y
x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
y = data.loc[cond, "colors.g-r.color.value"]
ax[1].text(xTxt, yTxt, f"{len(data[cond]):,d} SSOs", transform=ax[1].transAxes)

# Convert to PanSTARRS
dmag_g = color_conv_ztf_panstarrs("g", x)
dmag_r = color_conv_ztf_panstarrs("r", x)
x2 = x - (dmag_g - dmag_r)

y_sdss = color_conv_skymapper_sdss("g-r", y)
dmag_g = color_conv_panstarrs_sdss("g", data.loc[cond, "colors.g-i.color.value"])
dmag_r = color_conv_panstarrs_sdss("r", data.loc[cond, "colors.g-i.color.value"])
y2 = y_sdss + (dmag_g - dmag_r)

# Plot histogram
ax[1].hist2d(
    x2, y2, range=r, bins=b, cmap="hot_r", vmin=vmin, vmax=vmax, rasterized=True
)


# ----------------------------------------------------------------------
# Axes
ax[0].set_ylabel("SDSS g-r")
ax[1].set_ylabel("SkyMapper g-r")
ax[1].set_xlabel("ZTF H$_g$-H$_r$")
                 
xx = np.linspace(0, 1, num=10)
for a in ax[:].ravel():
    a.set_xlim(r[0][0], r[0][1])
    a.set_ylim(r[1][0], r[1][1])
    a.plot(xx, xx, linestyle="dashed", color="grey")
    a.grid()


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_comparison_vsplit.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "color_comparison_vsplit.pgf"),
)
plt.close()

In [23]:
fig, ax = plt.subplots(1, 2,
    figsize=(16, 8),
    sharex=True,
    sharey=True,
    gridspec_kw={"wspace": 0.02, "hspace": 0.02},
)


for a in ax:
    a.grid()

complexes = ["E", "V", "Q", "A", "S", "L", "K", "D", "M", "Z", "C", "B", "Ch", "P"]
complexes = ["V", "Q", "A", "S", "D", "Z", "C", "B"]
markers = ['v','o','^','.','d','P','s','<']
for i, c in enumerate(complexes):
    cond = (
        maskFINK
        & (data["taxonomy.complex"] == c)
        & ((data["SHG1G2_dH_g"] + data["SHG1G2_dH_r"]) < 0.3)
        & (data["colors.i-z.facility"] == "SDSS")
        & (data["colors.i-z.color.value"].notna())
    )
    x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
    y = data.loc[cond, "colors.i-z.color.value"]
    print( c, len(data[cond]))
    ax[0].scatter(x, y, label=c, marker=markers[i] )


    ax[0].text(x.mean(), y.mean(), c, ha='center', va='center', color='black' )
    # axs[i].text( G1_m, G2_m, c, ha='center', va='center', color='white' )


ax[0].legend()
ax[0].set_xlabel("Hg-Hr / ZTF")
ax[0].set_ylabel("i-z / SDSS")
ax[0].set_xlim(-0.1, 1.1)
ax[0].set_ylim(-0.7, 0.5)


for i, c in enumerate(complexes):
    cond = (
        maskFINK
        & (data["taxonomy.complex"] == c)
        & ((data["SHG1G2_dH_g"] + data["SHG1G2_dH_r"]) < 0.3)
        & (data["colors.i-z.facility"] == "SMSS")
        & (data["colors.i-z.color.value"].notna())
    )
    x = data.loc[cond, "SHG1G2_H_g"] - data.loc[cond, "SHG1G2_H_r"]
    y = data.loc[cond, "colors.i-z.color.value"]
    print( c, len(data[cond]))
    ax[1].scatter(x, y, label=c, marker=markers[i] )


    ax[1].text(x.mean(), y.mean(), c, ha='center', va='center', color='black' )
    # axs[i].text( G1_m, G2_m, c, ha='center', va='center', color='white' )


ax[1].legend()
ax[1].set_xlabel("Hg-Hr / ZTF")
ax[1].set_ylabel("i-z / SkyMapper")
ax[1].set_xlim(-0.1, 1.1)
ax[1].set_ylim(-0.7, 0.5)


V 182
Q 13
A 27
S 1461
D 113
Z 1
C 673
B 156
V 361
Q 43
A 133
S 1720
D 215
Z 10
C 620
B 317


(-0.7, 0.5)